In [1]:
import pandas as pd
pd.set_option('display.max_columns', 100)

userId - this is host_id

In [2]:
raw_data_path = '../../data/raw_data/raw_data_Anna.csv'
df = pd.read_csv(raw_data_path)

print(f"{df.shape=}")
df.head()

df.shape=(2399, 25)


,id,userId,name,address,city,isSuperhost,lat,lng,persons,rating,reviewsCount,type,cancelPolicy,deeplink,hostThumbnail,price_currency,price_rate,price_total,bathrooms,bedrooms,beds,previewAmenities,url,images,amenityIds
0,36322781,22847334,Cozy Comfy Guest Suite (Nanaimo Skytrain),"Vancouver, BC, Canada",Vancouver,True,49.25094,-123.05199,2,4.91,283,Entire guest suite,CANCEL_MODERATE,https://www.airbnb.com/rooms/36322781?check_in...,https://a0.muscache.com/im/pictures/user/75bc6...,CAD,174,696,1.0,1,1,"Free parking, Wifi, Kitchen, Self check-in",https://www.airbnb.com/rooms/36322781,['https://a0.muscache.com/im/pictures/521551d8...,"[1, 4, 8, 9, 522, 77, 79, 657, 85, 86, 23, 89,..."
1,51320938,30084673,"NEW, clean, modern, private suite","Vancouver, BC, Canada",Vancouver,True,49.25369,-123.09964,2,4.92,107,Entire home,CANCEL_BETTER_STRICT_WITH_GRACE_PERIOD,https://www.airbnb.com/rooms/51320938?check_in...,https://a0.muscache.com/im/pictures/user/27680...,CAD,224,896,1.0,1,2,"Wifi, Kitchen, Self check-in",https://www.airbnb.com/rooms/51320938,['https://a0.muscache.com/im/pictures/400ac6fc...,"[1, 33, 34, 35, 4, 100, 8, 40, 41, 44, 45, 51,..."
2,1152216438211720822,57959028,Pet friendly Gem in Gastown,"Vancouver, BC, Canada",Vancouver,True,49.28272,-123.10362,3,0.00,0,Entire rental unit,CANCEL_FLEXIBLE,https://www.airbnb.com/rooms/11522164382117208...,https://a0.muscache.com/im/pictures/user/User-...,CAD,312,1248,1.0,1,1,"Wifi, Kitchen",https://www.airbnb.com/rooms/1152216438211720822,['https://a0.muscache.com/im/pictures/miso/Hos...,"[1, 33, 35, 99, 179, 4, 36, 37, 8, 12, 47]"
3,34203078,258206915,Downtown Vancouver Coal Harbour w/ parking,"Vancouver, BC, Canada",Vancouver,False,49.28618,-123.12312,4,4.76,369,Entire condo,CANCEL_MODERATE,https://www.airbnb.com/rooms/34203078?check_in...,https://a0.muscache.com/im/pictures/user/3325b...,CAD,469,1873,1.0,1,3,"Free parking, Wifi, Kitchen, Self check-in",https://www.airbnb.com/rooms/34203078,['https://a0.muscache.com/im/pictures/eaee26aa...,"[1, 4, 5, 8, 9, 137, 10, 522, 657, 21, 663, 66..."
4,21356520,11824155,Private and spacious room in upper floor,"Vancouver, BC, Canada",Vancouver,True,49.23965,-123.04831,2,4.90,112,Private room in home,CANCEL_MODERATE,https://www.airbnb.com/rooms/21356520?check_in...,https://a0.muscache.com/im/pictures/user/eeb58...,CAD,107,427,1.0,1,1,"Free parking, Wifi",https://www.airbnb.com/rooms/21356520,['https://a0.muscache.com/im/pictures/edcf7494...,"[1, 129, 2, 4, 9, 73, 522, 77, 79, 146, 85, 86..."


expected_output is what we would like to get finally given host_id (userId column in raw_data)

In [3]:
# Given userId = 410216127 in raw_data.csv, we need to get:

expected_output = {
  "host_id": "410216127",
  "host_name": "Avani And Krunal",
  "isSuperhost": True,
  "reviews": 627,
  "rating": 4.96,
  "yearsHosting": 2,
  "location": "Delta, Canada",
  "profilePictureUrl": "https://a0.muscache.com/im/pictures/user/User-410216127/original/faaa0e7b-4242-4687-bb82-aff68e029400.jpeg?aki_policy=profile_large",

  "listings": [
      {
        "listing_id": "941828367326510565",
        "description": "Home\nBedroom with Private bath"
      },
      {
        "listing_id": "808448380230731877",
        "description": "Guest suite\nModern Suite in Delta"
      },
      {
        "listing_id": "829231885851222725",
        "description": "Guest suite\nPrivate Guest Suite w/separate entrance."
      },
      {
        "listing_id": "941818690448679242",
        "description": "Home\nBedroom with private bath & patio"
      },
      {
        "listing_id": "677838844811924690",
        "description": "Home\nBrandnew Luxurious 3-Bedroom Home"
      },
      {
        "listing_id": "874531542306878126",
        "description": "Guest suite\nStudio Suite w/private entrance"
      },
      {
        "listing_id": "1131552179220065444",
        "description": "Other\nA perfect place for celebrations in Downtown"
      },
      {
        "listing_id": "941202125287004383",
        "description": "Home\nMaster Bedroom with Private Bath"
      }
    ]
}

In [4]:
import requests
from bs4 import BeautifulSoup

def extract_airbnb_host_info(url: str) -> dict:
  """Extracts host information and listings from an Airbnb host page.

  Args:
    url: The URL of the Airbnb host page.

  Returns:
    A dictionary containing the host information and listings.
  """

  response = requests.get(url)
  response.raise_for_status()  # Raise an exception for bad status codes

  soup = BeautifulSoup(response.content, 'html.parser')

  # Host information
  host_data = {}

  # Extract profile picture URL
  profile_picture_element = soup.find('img', {'data-original-uri': True})
  host_data['profilePictureUrl'] = profile_picture_element['data-original-uri'] if profile_picture_element else None

  # Extract host name
  host_name_element = soup.find('h2', {'elementtiming': 'LCP-target'})
  if host_name_element:
    host_data['name'] = host_name_element.text.strip()

  # Extract Superhost status
  superhost_element = soup.find('span', {'class': 's2nv573'})
  host_data['isSuperhost'] = True if superhost_element else False

  # Extract review count and rating
  review_count_element = soup.find('span', {'class': 'a8jt5op', 'data-testid': 'Reviews-stat-heading'})
  host_data['reviews'] = int(review_count_element.text.strip()) if review_count_element else None

  rating_element = soup.find('span', {'class': 'a8jt5op', 'data-testid': 'Rating-stat-heading'})
  host_data['rating'] = float(rating_element.text.strip().split()[0]) if rating_element else None

  # Extract years hosting
  years_hosting_element = soup.find('span', {'class': 'a8jt5op', 'data-testid': 'Years hosting-stat-heading'})
  host_data['yearsHosting'] = int(years_hosting_element.text.strip()) if years_hosting_element else None

  # Extract location
  location_element = soup.find('span', {'class': 'i1slxodh', 'data-testid': 'listing-card-subtitle'})
  host_data['location'] = location_element.text.strip() if location_element else None

  # Listings
  listings = []
  listing_elements = soup.find_all('div', {'aria-labelledby': True, 'data-testid': 'card-container'})
  for listing_element in listing_elements:
    listing = {}
    listing['listing_id'] = listing_element['aria-labelledby'].split('_')[-1]
    listing['description'] = listing_element.find('div', {'class': 'fb4nyux'}).text.strip()
    listings.append(listing)

  host_data['listings'] = listings

  return host_data



In [5]:
# Example with one listing
user_id = 504066776 
host_url = f'https://www.airbnb.ca/users/show/{user_id}'
print(f"{host_url=}")

host_info = extract_airbnb_host_info(host_url)

host_info

host_url='https://www.airbnb.ca/users/show/504066776'


{'profilePictureUrl': 'https://a0.muscache.com/im/pictures/user/User-504066776/original/712ea262-1298-4715-a1b5-1868dc8f2790.jpeg?aki_policy=profile_large',
 'name': 'Maye',
 'isSuperhost': True,
 'reviews': None,
 'rating': None,
 'yearsHosting': None,
 'location': None,
 'listings': [{'listing_id': '1034049419047464774',
   'description': "Comfy Room @ YVR -Pet Lover's Oasis Lic# 23 031530Comfy Room @ YVR -Pet Lover's Oasis Lic# 23 031530"},
  {'listing_id': '840818841362172391',
   'description': 'Spacious Room YVR Lic 23-031530Spacious Room YVR Lic 23-031530'}]}

In [6]:
# Example with multiple listings
user_id = 410216127 
host_url = f"https://www.airbnb.ca/users/show/{user_id}"
print(f"{host_url=}")

host_info = extract_airbnb_host_info(host_url)

host_info

host_url='https://www.airbnb.ca/users/show/410216127'


{'profilePictureUrl': 'https://a0.muscache.com/im/pictures/user/User-410216127/original/faaa0e7b-4242-4687-bb82-aff68e029400.jpeg?aki_policy=profile_large',
 'name': 'Avani And Krunal',
 'isSuperhost': True,
 'reviews': None,
 'rating': None,
 'yearsHosting': None,
 'location': None,
 'listings': [{'listing_id': '941202125287004383',
   'description': 'Master Bedroom with Private BathMaster Bedroom with Private Bath'},
  {'listing_id': '829231885851222725',
   'description': 'Private Guest Suite w/separate entrance.Private Guest Suite w/separate entrance.'},
  {'listing_id': '874531542306878126',
   'description': 'Studio Suite w/private entranceStudio Suite w/private entrance'},
  {'listing_id': '941818690448679242',
   'description': 'Bedroom with private bath & patioBedroom with private bath & patio'},
  {'listing_id': '941828367326510565',
   'description': 'Bedroom with Private bathBedroom with Private bath'},
  {'listing_id': '808448380230731877',
   'description': 'Modern Suite 